# Project 6: Analyzing Stock Sentiment from Twits
## Instructions
Each problem consists of a function to implement and instructions on how to implement the function.  The parts of the function that need to be implemented are marked with a `# TODO` comment.

## Packages
When you implement the functions, you'll only need to you use the packages you've used in the classroom, like [Pandas](https://pandas.pydata.org/) and [Numpy](http://www.numpy.org/). These packages will be imported for you. We recommend you don't add any import statements, otherwise the grader might not be able to run your code.

### Load Packages

In [30]:
import json
import nltk
import os
import random
import re
import torch

from torch import nn, optim
import torch.nn.functional as F

## Introduction
When deciding the value of a company, it's important to follow the news. For example, a product recall or natural disaster in a company's product chain. You want to be able to turn this information into a signal. Currently, the best tool for the job is a Neural Network. 

For this project, you'll use posts from the social media site [StockTwits](https://en.wikipedia.org/wiki/StockTwits). The community on StockTwits is full of investors, traders, and entrepreneurs. Each message posted is called a Twit. This is similar to Twitter's version of a post, called a Tweet. You'll build a model around these twits that generate a sentiment score.

We've collected a bunch of twits, then hand labeled the sentiment of each. To capture the degree of sentiment, we'll use a five-point scale: very negative, negative, neutral, positive, very positive. Each twit is labeled -2 to 2 in steps of 1, from very negative to very positive respectively. You'll build a sentiment analysis model that will learn to assign sentiment to twits on its own, using this labeled data.

The first thing we should to do, is load the data.

## Import Twits 
### Load Twits Data 
This JSON file contains a list of objects for each twit in the `'data'` field:

```
{'data':
  {'message_body': 'Neutral twit body text here',
   'sentiment': 0},
  {'message_body': 'Happy twit body text here',
   'sentiment': 1},
   ...
}
```

The fields represent the following:

* `'message_body'`: The text of the twit.
* `'sentiment'`: Sentiment score for the twit, ranges from -2 to 2 in steps of 1, with 0 being neutral.


To see what the data look like by printing the first 10 twits from the list. 

In [31]:
# from shutil import copyfile
# copyfile(os.path.join('..', '..', 'data', 'project_6_stocktwits', 'twits.json'), 'twits.json')
with open(os.path.join('..', '..', 'data', 'project_6_stocktwits', 'twits.json'), 'r') as f:
    twits = json.load(f)

print(twits['data'][:10])

[{'message_body': '$FITB great buy at 26.00...ill wait', 'sentiment': 2, 'timestamp': '2018-07-01T00:00:09Z'}, {'message_body': '@StockTwits $MSFT', 'sentiment': 1, 'timestamp': '2018-07-01T00:00:42Z'}, {'message_body': '#STAAnalystAlert for $TDG : Jefferies Maintains with a rating of Hold setting target price at USD 350.00. Our own verdict is Buy  http://www.stocktargetadvisor.com/toprating', 'sentiment': 2, 'timestamp': '2018-07-01T00:01:24Z'}, {'message_body': '$AMD I heard there’s a guy who knows someone who thinks somebody knows something - on StockTwits.', 'sentiment': 1, 'timestamp': '2018-07-01T00:01:47Z'}, {'message_body': '$AMD reveal yourself!', 'sentiment': 0, 'timestamp': '2018-07-01T00:02:13Z'}, {'message_body': '$AAPL Why the drop? I warren Buffet taking out his position?', 'sentiment': 1, 'timestamp': '2018-07-01T00:03:10Z'}, {'message_body': '$BA bears have 1 reason on 06-29 to pay more attention https://dividendbot.com?s=BA', 'sentiment': -2, 'timestamp': '2018-07-01T

### Length of Data
Now let's look at the number of twits in dataset. Print the number of twits below.

In [32]:
"""print out the number of twits"""

# TODO Implement 
print(len(twits['data']))

1548010


### Split Message Body and Sentiment Score

In [33]:
messages = [twit['message_body'] for twit in twits['data']]
# Since the sentiment scores are discrete, we'll scale the sentiments to 0 to 4 for use in our network
sentiments = [twit['sentiment'] + 2 for twit in twits['data']]

## Preprocessing the Data
With our data in hand we need to preprocess our text. These twits are collected by filtering on ticker symbols where these are denoted with a leader $ symbol in the twit itself. For example,

`{'message_body': 'RT @google Our annual look at the year in Google blogging (and beyond) http://t.co/sptHOAh8 $GOOG',
 'sentiment': 0}`

The ticker symbols don't provide information on the sentiment, and they are in every twit, so we should remove them. This twit also has the `@google` username, again not providing sentiment information, so we should also remove it. We also see a URL `http://t.co/sptHOAh8`. Let's remove these too.

The easiest way to remove specific words or phrases is with regex using the `re` module. You can sub out specific patterns with a space:

```python
re.sub(pattern, ' ', text)
```
This will substitute a space with anywhere the pattern matches in the text. Later when we tokenize the text, we'll split appropriately on those spaces.

### Pre-Processing

In [34]:
nltk.download('wordnet')

def preprocess(message):
    """
    This function takes a string as input, then performs these operations: 
        - lowercase
        - remove URLs
        - remove ticker symbols 
        - removes punctuation
        - tokenize by splitting the string on whitespace 
        - removes any single character tokens
    
    Parameters
    ----------
        message : The text message to be preprocessed.
        
    Returns
    -------
        tokens: The preprocessed text into tokens.
    """ 
    #TODO: Implement 
    
    # Lowercase the twit message
    text = message.lower()
    
    # Replace URLs with a space in the message
    #text = re.sub(r' http://.+ ', ' ', message)
    text = re.sub(r'https?://.*[\r\n]*', ' ', text)
    
    # Replace ticker symbols with a space. The ticker symbols are any stock symbol that starts with $.
    regex = re.compile(r'\$\w+')
    text = regex.sub(' ', text)
    
    # Replace StockTwits usernames with a space. The usernames are any word that starts with @.
    regex = re.compile(r'@\w+')
    text = regex.sub(' ', text)

    # Replace everything not a letter with a space
    regex = re.compile(r'(\W|\d)')
    text = regex.sub(' ', text)
    
    # Tokenize by splitting the string on whitespace into a list of words
    tokens = text.split()

    # Lemmatize words using the WordNetLemmatizer. You can ignore any word that is not longer than one character.
    wnl = nltk.stem.WordNetLemmatizer()
    tokens = [wnl.lemmatize(_, pos='v') for _ in tokens if len(_)>1 ]
    tokens = [wnl.lemmatize(_) for _ in tokens]
    
    assert type(tokens) == list, 'Tokens should be list'
    return tokens

#test_text = 'RT @google Our annual look at the year in Google blogging (and beyond) 100.25 http://t.co/sptHOAh8 $GOOG'
#print(preprocess(test_text))

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


>Note: You must ensure that after preprocessing the text should NOT include:
- Numbers
- URLs
- Single character tokens
- Ticker symbols (these should be removed even if they don't appear at the beginning)

### Preprocess All the Twits 
Now we can preprocess each of the twits in our dataset. Apply the function `preprocess` to all the twit messages.

In [35]:
# TODO Implement
tokenized = []
for message in messages[:800000]:
    tocken = preprocess(message)
    tokenized.append(tocken)
print(tokenized[:5])

[['great', 'buy', 'at', 'ill', 'wait'], [], ['staanalystalert', 'for', 'jefferies', 'maintain', 'with', 'rat', 'of', 'hold', 'set', 'target', 'price', 'at', 'usd', 'our', 'own', 'verdict', 'be', 'buy'], ['hear', 'there', 'guy', 'who', 'know', 'someone', 'who', 'think', 'somebody', 'know', 'something', 'on', 'stocktwits'], ['reveal', 'yourself']]


### Bag of Words
Now with all of our messages tokenized, we want to create a vocabulary and count up how often each word appears in our entire corpus. Use the [`Counter`](https://docs.python.org/3.1/library/collections.html#collections.Counter) function to count up all the tokens.

In [36]:
from collections import Counter

"""
Create a vocabulary by using Bag of words
"""
bow = Counter()
for tokens in tokenized:
    bow.update(tokens)
vocab = sorted(bow, key=bow.get, reverse=True)
vocab_to_int = {word: ii for ii, word in enumerate(vocab, 1)}
int_to_vocab = {ii: word for word, ii in vocab_to_int.items()}
# print(vocab_to_int)
# print(int_to_vocab)

### Frequency of Words Appearing in Message
With our vocabulary, now we'll remove some of the most common words such as 'the', 'and', 'it', etc. These words don't contribute to identifying sentiment and are really common, resulting in a lot of noise in our input. If we can filter these out, then our network should have an easier time learning.

We also want to remove really rare words that show up in a only a few twits. Here you'll want to divide the count of each word by the **number of messages** calculated in the code block above (i.e. `len(messages))`. Then remove words that only appear in some small fraction of the messages.

>Note: There is not an exact number for low and high-frequency cut-offs, however there is a correct optimal range.
You should ideally set up low-frequency cut-off from 0.0000002 to 0.000007 (inclusive) and high-frequency from 5 to 20 (inclusive). If the number is too big, we lose lots of important words that we can use in our data.

In [37]:
import numpy as np

# Dictionart that contains the Frequency of words appearing in messages.
# The key is the token and the value is the frequency of that word in the corpus.
total_word_count = sum(bow.values())
freqs = {word: count/total_word_count for word, count in vocab_to_int.items()}

# Float that is the frequency cutoff. Drop words with a frequency that is lower or equal to this number.
low_cutoff = 0.0000002

# Integer that is the cut off for most common words. Drop words that are the `high_cutoff` most common words.
high_cutoff = 18

# The k most common words in the corpus. Use `high_cutoff` as the k.
K_most_common = [entry[0] for entry in bow.most_common(high_cutoff) ]

filtered_words = [word for word in freqs.keys() if (freqs[word] > low_cutoff and word not in K_most_common)]

print(bow.most_common(high_cutoff))
print(len(vocab_to_int),len(filtered_words))

[('be', 295107), ('the', 200754), ('to', 199473), ('for', 143127), ('on', 127635), ('of', 108026), ('and', 105843), ('in', 105548), ('this', 102037), ('it', 97532), ('at', 76037), ('will', 66082), ('up', 61542), ('report', 61148), ('buy', 60158), ('have', 54218), ('go', 54191), ('you', 47456)]
62280 62262


### Updating Vocabulary by Removing Filtered Words
Let's creat three variables that will help with our vocabulary.

In [38]:
"""
Set the following variables:
    vocab
    id2vocab
    filtered
"""

# A dictionary for the `filtered_words`. The key is the word and value is an id that represents the word.
vocab = {word: word_id for word_id, word in enumerate(filtered_words, 1)}
# Reverse of the `vocab` dictionary. The key is word id and value is the word.
id2vocab = {val: key for key, val in vocab.items()}
# tokenized with the words not in `filtered_words` removed.
filtered = [[token for token in message if token in vocab] for message in tokenized]
assert set(vocab.keys()) == set(id2vocab.values()), 'Check vocab and id2vocab dictionaries'

### Balancing the classes
Let's do a few last pre-processing steps. If we look at how our twits are labeled, we'll find that 50% of them are neutral. This means that our network will be 50% accurate just by guessing 0 every single time. To help our network learn appropriately, we'll want to balance our classes.
That is, make sure each of our different sentiment scores show up roughly as frequently in the data.

What we can do here is go through each of our examples and randomly drop twits with neutral sentiment. What should be the probability we drop these twits if we want to get around 20% neutral twits starting at 50% neutral? We should also take this opportunity to remove messages with length 0.

In [39]:
balanced = {'messages': [], 'sentiments':[]}
n_neutral = sum(1 for each in sentiments if each == 2)
N_examples = len(sentiments)
keep_prob = (N_examples - n_neutral)/4/n_neutral

for idx, sentiment in enumerate(sentiments[:len(tokenized)]):
    message = filtered[idx]
    if len(message) == 0:
        # skip this message because it has length zero
        continue
    elif sentiment != 2 or random.random() < keep_prob:
        balanced['messages'].append(message)
        balanced['sentiments'].append(sentiment)

# print(balanced['messages'][8:10] )
# print(balanced['sentiments'][8:10])
# print(len(balanced['messages'])==len(balanced['sentiments']))

If you did it correctly, you should see the following result 

In [40]:
n_neutral = sum(1 for each in balanced['sentiments'] if each == 2)
N_examples = len(balanced['sentiments'])
print(n_neutral/N_examples)

print(balanced['messages'][:10])
print(balanced['sentiments'][:10])

0.19778779776740532
[['great', 'ill', 'wait'], ['staanalystalert', 'jefferies', 'maintain', 'with', 'rat', 'hold', 'set', 'target', 'price', 'usd', 'our', 'own', 'verdict'], ['hear', 'there', 'guy', 'who', 'know', 'someone', 'who', 'think', 'somebody', 'know', 'something', 'stocktwits'], ['why', 'drop', 'warren', 'buffet', 'take', 'out', 'his', 'position'], ['bear', 'reason', 'pay', 'more', 'attention'], ['ok', 'good', 'we', 're', 'not', 'drop', 'price', 'over', 'weekend', 'lol'], ['daily', 'chart', 'we', 'need', 'get', 'back', 'above'], ['drop', 'per', 'week', 'after', 'spike', 'if', 'no', 'news', 'month', 'back', 'if', 'bo', 'then', 'bingo', 'what', 'odds'], ['strong'], ['short', 'ratio', 'short', 'float']]
[4, 4, 3, 3, 0, 3, 4, 0, 4, 0]


Finally let's convert our tokens into integer ids which we can pass to the network.

In [41]:
token_ids = [[vocab[word] for word in message] for message in balanced['messages']]
# token_freqs = [[vocab[word]/words_count for word in message] for message in balanced['messages']]
sentiments = balanced['sentiments']

## Neural Network
Now we have our vocabulary which means we can transform our tokens into ids, which are then passed to our network. So, let's define the network now!

Here is a nice diagram showing the network we'd like to build: 

#### Embed -> RNN -> Dense -> Softmax
### Implement the text classifier
Before we build text classifier, if you remember from the other network that you built in  "Sentiment Analysis with an RNN"  exercise  - which there, the network called " SentimentRNN", here we named it "TextClassifer" - consists of three main parts: 1) init function `__init__` 2) forward pass `forward`  3) hidden state `init_hidden`. 

This network is pretty similar to the network you built expect in the  `forward` pass, we use softmax instead of sigmoid. The reason we are not using sigmoid is that the output of NN is not a binary. In our network, sentiment scores have 5 possible outcomes. We are looking for an outcome with the highest probability thus softmax is a better choice.

In [42]:
class TextClassifier(nn.Module):
    def __init__(self, vocab_size, embed_size, lstm_size, output_size, lstm_layers=1, dropout=0.1):
        """
        Initialize the model by setting up the layers.

        Parameters
        ----------
            vocab_size : The vocabulary size.
            embed_size : The embedding layer size.
            lstm_size : The LSTM layer size.
            output_size : The output size.
            lstm_layers : The number of LSTM layers.
            dropout : The dropout probability.
        """
        super().__init__()
        self.vocab_size = vocab_size
        self.embed_size = embed_size
        self.lstm_size = lstm_size
        self.output_size = output_size
        self.lstm_layers = lstm_layers
        self.dropout = dropout

        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, lstm_size, lstm_layers,
                            dropout=dropout, batch_first=False)

        self.dropout = nn.Dropout(0.2)

        self.fc = nn.Linear(lstm_size, output_size)
        self.logsoftmax = nn.LogSoftmax(dim=1)

    def init_hidden(self, batch_size):
        """
        Initializes hidden state

        Parameters
        ----------
            batch_size : The size of batches.

        Returns
        -------
            hidden_state

        """
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        hidden = (weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_(),
                  weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_())
        return hidden

    def forward(self, nn_input, hidden_state):
        """
        Perform a forward pass of our model on nn_input.

        Parameters
        ----------
            nn_input : The batch of input to the NN.
            hidden_state : The LSTM hidden state.

        Returns
        -------
            logps: log softmax output
            hidden_state: The new hidden state.

        """
        batch_size = nn_input.size(0)
        embed_output = self.embedding(nn_input)
        
        lstm_output, hidden_state = self.lstm(embed_output, hidden_state)
        lstm_output = lstm_output[-1, :, :]
        
        drop_output = self.dropout(lstm_output)
        # drop_output = drop_output.contiguous().view(-1, self.lstm_size)
        
        liner_output = self.fc(drop_output)
        log_output = self.logsoftmax(liner_output)
        
        return log_output, hidden_state

### View Model

In [43]:
model = TextClassifier(len(vocab), 10, 6, 5, dropout=0.1, lstm_layers=2)
model.embedding.weight.data.uniform_(-1, 1)
input = torch.randint(0, 1000, (5, 4), dtype=torch.int64)
hidden = model.init_hidden(4)
logps, _ = model.forward(input, hidden)
print(np.exp(logps)

tensor([[-1.4230, -1.6054, -1.4891, -1.6034, -2.0294],
        [-1.3391, -1.6787, -1.4412, -1.6563, -2.0889],
        [-1.3338, -1.5636, -1.5158, -1.7188, -2.0540],
        [-1.5040, -1.5092, -1.5465, -1.6173, -1.9294]])


## Training
### DataLoaders and Batching
Now we should build a generator that we can use to loop through our data. It'll be more efficient if we can pass our sequences in as batches. Our input tensors should look like `(sequence_length, batch_size)`. So if our sequences are 40 tokens long and we pass in 25 sequences, then we'd have an input size of `(40, 25)`.

If we set our sequence length to 40, what do we do with messages that are more or less than 40 tokens? For messages with fewer than 40 tokens, we will pad the empty spots with zeros. We should be sure to **left** pad so that the RNN starts from nothing before going through the data. If the message has 20 tokens, then the first 20 spots of our 40 long sequence will be 0. If a message has more than 40 tokens, we'll just keep the first 40 tokens.

In [44]:
def dataloader(messages, labels, sequence_length=30, batch_size=32, shuffle=False):
    """
    Build a dataloader.
    """
    if shuffle:
        indices = list(range(len(messages)))
        random.shuffle(indices)
        messages = [messages[idx] for idx in indices]
        labels = [labels[idx] for idx in indices]

    total_sequences = len(messages)
    odd_lenth = total_sequences % (sequence_length * batch_size)
    total_sequences = total_sequences - odd_lenth

    for ii in range(0, total_sequences, batch_size):
        batch_messages = messages[ii: ii + batch_size]

        # First initialize a tensor of all zeros
        batch = torch.zeros((sequence_length, len(batch_messages)), dtype=torch.int64)
        for batch_num, tokens in enumerate(batch_messages):
            token_tensor = torch.tensor(tokens)
            # Left pad!
            start_idx = max(sequence_length - len(token_tensor), 0)
            batch[start_idx:, batch_num] = token_tensor[:sequence_length]

        label_tensor = torch.tensor(labels[ii: ii + len(batch_messages)])

        yield batch, label_tensor

### Training and  Validation
With our data in nice shape, we'll split it into training and validation sets.

In [45]:
"""
Split data into training and validation datasets. Use an appropriate split size.
The features are the `token_ids` and the labels are the `sentiments`.
"""   

valid_split = int(len(token_ids)*0.9)
train_features = token_ids[:valid_split]  
valid_features = token_ids[valid_split:]
train_labels = sentiments[:valid_split] 
valid_labels = sentiments[valid_split:]

In [46]:
text_batch, labels = next(iter(dataloader(train_features, train_labels, sequence_length=20, batch_size=64)))
model = TextClassifier(len(vocab)+1, 200, 128, 5, dropout=0.)
hidden = model.init_hidden(64)
logps, hidden = model.forward(text_batch, hidden)
top_p, top_class = logps.topk(1, dim=1)
# print(top_class.view(labels.shape))
# print(labels)
# print(top_class==labels)
print(text_batch[:,:5], labels[:5])
print(train_features[:5], train_labels[:5])
#mean_message = [[id2vocab[idx] for idx in idxs] for idxs in train_features[:5]]
#print(mean_message)

tensor([[    0,     0,     0,     0,     0],
        [    0,     0,     0,     0,     0],
        [    0,     0,     0,     0,     0],
        [    0,     0,     0,     0,     0],
        [    0,     0,     0,     0,     0],
        [    0,     0,     0,     0,     0],
        [    0,     0,     0,     0,     0],
        [    0,   543,     0,     0,     0],
        [    0,  1238,   562,     0,     0],
        [    0,   939,   101,     0,     0],
        [    0,    13,   186,     0,     0],
        [    0,   143,   149,     0,     0],
        [    0,    49,   120,   122,     0],
        [    0,   241,   445,   118,     0],
        [    0,   154,   149,  1324,     0],
        [    0,    47,    72,  1057,    71],
        [    0,   647,  1338,    74,   301],
        [  139,   211,   120,    24,   214],
        [ 1509,   225,   387,   373,    25],
        [  134,   579,  1096,   121,   550]]) tensor([ 4,  4,  3,  3,  0])
[[139, 1509, 134], [543, 1238, 939, 13, 143, 49, 241, 154, 47, 647, 21

### Training
It's time to train the neural network!

In [79]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = TextClassifier(len(vocab)+1, 1024, 512, 5, lstm_layers=2, dropout=0.5)
model.embedding.weight.data.uniform_(-1, 1)
model.to(device)

TextClassifier(
  (embedding): Embedding(62263, 1024)
  (lstm): LSTM(1024, 512, num_layers=2, dropout=0.5)
  (dropout): Dropout(p=0.2)
  (fc): Linear(in_features=512, out_features=5, bias=True)
  (logsoftmax): LogSoftmax()
)

In [80]:
def train(epochs, batch_size, learning_rate):
    print_every = 100
    criterion = nn.NLLLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    model.train()

    for epoch in range(epochs):
        print('Starting epoch {}'.format(epoch + 1))
        steps = 0
        train_loss = 0

        hidden = model.init_hidden(batch_size)
        for text_batch, labels in dataloader(
                train_features, train_labels, batch_size=batch_size, sequence_length=20, shuffle=True):
            steps += 1
            # Set Device
            # text_batch, labels = torch.LongTensor(text_batch), torch.LongTensor(labels)
            text_batch, labels = text_batch.to(device), labels.to(device)
            for each in hidden:
                each.to(device)

            model.zero_grad()
            hidden = tuple([each.data for each in hidden])

            log_output, hidden = model.forward(text_batch, hidden)
            loss = criterion(log_output.squeeze(), labels.long())
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), 5)
            optimizer.step()

            train_loss += loss.item()

            if steps % print_every == 0:
                valid_loss = []
                model.eval()
                accuracy = 0
                val_hidden = model.init_hidden(batch_size)
                for val_batch, val_labels in dataloader(
                        valid_features, valid_labels, batch_size=batch_size, sequence_length=20, shuffle=True):
                    val_hidden = tuple([each.data for each in val_hidden])

                    # val_batch, val_labels = torch.LongTensor(val_batch), torch.LongTensor(val_labels)
                    val_batch, val_labels = val_batch.to(device), val_labels.to(device)
                    for each in hidden:
                        each.to(device)

                    val_out, val_hidden = model.forward(val_batch, val_hidden)
                    val_loss = criterion(val_out.squeeze(), val_labels.long())
                    val_p, val_class = val_out.topk(1, dim=1)
                    equal = val_class.view(val_labels.shape) == val_labels
                    accuracy += torch.mean(equal.type(torch.FloatTensor)).item()

                    valid_loss.append(val_loss.item())

                model.train()
                print(f"steps {steps}.. "
                      f"Train loss: {train_loss/print_every:.3f}.. "
                      f"valid loss: {np.mean(valid_loss):.3f}.. "
                      f"valid accuracy: {accuracy/(len(valid_labels)//batch_size):.3f}")
                
                train_loss = 0

epochs = 3
batch_size = 256
learning_rate = 0.001
train(epochs=epochs, batch_size=batch_size, learning_rate=learning_rate)

Starting epoch 1
steps 100.. Train loss: 1.230.. valid loss: 1.130.. valid accuracy: 0.513
steps 200.. Train loss: 1.023.. valid loss: 1.044.. valid accuracy: 0.558
steps 300.. Train loss: 0.967.. valid loss: 0.996.. valid accuracy: 0.578
steps 400.. Train loss: 0.925.. valid loss: 0.965.. valid accuracy: 0.591
steps 500.. Train loss: 0.917.. valid loss: 0.955.. valid accuracy: 0.597
steps 600.. Train loss: 0.893.. valid loss: 0.937.. valid accuracy: 0.605
steps 700.. Train loss: 0.880.. valid loss: 0.918.. valid accuracy: 0.616
steps 800.. Train loss: 0.870.. valid loss: 0.922.. valid accuracy: 0.612
steps 900.. Train loss: 0.857.. valid loss: 0.906.. valid accuracy: 0.616
steps 1000.. Train loss: 0.852.. valid loss: 0.901.. valid accuracy: 0.624
steps 1100.. Train loss: 0.839.. valid loss: 0.895.. valid accuracy: 0.623
steps 1200.. Train loss: 0.848.. valid loss: 0.888.. valid accuracy: 0.625
steps 1300.. Train loss: 0.837.. valid loss: 0.891.. valid accuracy: 0.628
steps 1400.. Trai

In [81]:
#save model
model.to('cpu')
model_name = 'analyzing_stock_sentiment_rnn.net'

checkpoint = {'embed_size':1024,
              'outpu_size':5,
              'lstm_size': model.lstm_size,
              'lstm_layers': model.lstm_layers,
              'state_dict': model.state_dict(),
              'vocab_size': 62263,
              'batch_size': 256}

with open(model_name, 'wb') as f:
    torch.save(checkpoint, f)

In [57]:
# load model from file
import torch
with open('analyzing_stock_sentiment_rnn.net', 'rb') as f:
    checkpoint = torch.load(f)

model = TextClassifier(checkpoint['vocab_size'], 1024, 512, 5, lstm_layers=2, dropout=0.2)
model.load_state_dict(checkpoint['state_dict'])

## Making Predictions
### Prediction 
Okay, now that you have a trained model, try it on some new twits and see if it works appropriately. Remember that for any new text, you'll need to preprocess it first before passing it to the network. Implement the `predict` function to generate the prediction vector from a message.

In [82]:
def predict(text, model, vocab):
    """
    Make a prediction on a single sentence.

    Parameters
    ----------
        text : The string to make a prediction on.
        model : The model to use for making the prediction.
        vocab : Dictionary for word to word ids. The key is the word and the value is the word id.

    Returns
    -------
        pred : Prediction vector
    """
    tokens = preprocess(text)

    # Filter non-vocab words
    tokens = [token for token in tokens if token in vocab]

    # Convert words to ids
    tokens = [vocab[word] for word in tokens]

    # Adding a batch dimension
    token_tensor = torch.tensor(tokens)
    batch = torch.zeros((20, 1), dtype=torch.int64)
    start_idx = max(20 - len(token_tensor), 0)
    batch[start_idx:, 0] = token_tensor[:20]
    text_input = batch
    # Get the NN output
    hidden = model.init_hidden(1)
    logps, _ = model.forward(text_input, hidden)
    
    # Take the exponent of the NN output to get a range of 0 to 1 for each label.
    pred = logps.detach().numpy().squeeze()
    pred = np.exp(pred)
    
    top_p, top_class = logps.topk(1)
    print(top_class)
    return pred

In [83]:
text = "Google is working on self driving cars, I'm bullish high value on $goog"
model.eval()
model.to("cpu")
predict(text, model, vocab)

tensor([[ 3]])


array([  8.66124610e-05,   5.59147075e-03,   3.05654784e-03,
         8.58421266e-01,   1.32844076e-01], dtype=float32)

### Questions: What is the prediction of the model? What is the uncertainty of the prediction?

The prediction of model is most probably postive. 
We can't certain the result is correct and if it is postive or very postive.

Now we have a trained model and we can make predictions. We can use this model to track the sentiments of various stocks by predicting the sentiments of twits as they are coming in. Now we have a stream of twits. For each of those twits, pull out the stocks mentioned in them and keep track of the sentiments. Remember that in the twits, ticker symbols are encoded with a dollar sign as the first character, all caps, and 2-4 letters, like $AAPL. Ideally, you'd want to track the sentiments of the stocks in your universe and use this as a signal in your larger model(s).

## Testing
### Load the Data 

In [84]:
with open(os.path.join('..', '..', 'data', 'project_6_stocktwits', 'test_twits.json'), 'r') as f:
    test_data = json.load(f)

### Twit Stream

In [85]:
def twit_stream():
    for twit in test_data['data']:
        yield twit

next(twit_stream())

{'message_body': '$JWN has moved -1.69% on 10-31. Check out the movement and peers at  https://dividendbot.com?s=JWN',
 'timestamp': '2018-11-01T00:00:05Z'}

Using the `prediction` function, let's apply it to a stream of twits.

In [86]:
def score_twits(stream, model, vocab, universe):
    """ 
    Given a stream of twits and a universe of tickers, return sentiment scores for tickers in the universe.
    """
    for twit in stream:

        # Get the message text
        text = twit['message_body']
        symbols = re.findall('\$[A-Z]{2,4}', text)
        score = predict(text, model, vocab)

        for symbol in symbols:
            if symbol in universe:
                yield {'symbol': symbol, 'score': score, 'timestamp': twit['timestamp']}

In [87]:
universe = {'$BBRY', '$AAPL', '$AMZN', '$BABA', '$YHOO', '$LQMT', '$FB', '$GOOG', '$BBBY', '$JNUG', '$SBUX', '$MU'}
score_stream = score_twits(twit_stream(), model, vocab, universe)

next(score_stream)

tensor([[ 2]])
tensor([[ 1]])
tensor([[ 4]])


{'symbol': '$AAPL',
 'score': array([ 0.15890795,  0.01116017,  0.05918332,  0.07029199,  0.70045656], dtype=float32),
 'timestamp': '2018-11-01T00:00:18Z'}

That's it. You have successfully built a model for sentiment analysis! 

## Submission
Now that you're done with the project, it's time to submit it. Click the submit button in the bottom right. One of our reviewers will give you feedback on your project with a pass or not passed grade. You can continue to the next section while you wait for feedback.